In [ ]:
# !pip install openai==0.28

In [ ]:
## GPT 3.5

In [1]:
import pandas as pd
import openai

In [2]:
openai.api_key_path = 'openai-key.txt'

In [3]:
df = pd.read_csv('result.csv')

general_prompts_df = pd.read_csv('dataset/general-prompts.csv')

prompt_engineered_df = pd.read_csv('dataset/prompt-engineered.csv')

In [ ]:
def get_response(prompt):
    """
    Function to get the response from the model for a given prompt.
    """
    response = openai.Completion.create(
        engine="text-davinci-002",  # Select the appropriate model
        prompt=prompt,
        max_tokens=150  # Adjust as needed
    )
    return response.choices[0].text.strip()

# Example usage with one question from each set
question_original = df['question'][0]  
question_general = general_prompts_df['question'][0]  
question_engineered = prompt_engineered_df['question'][0]  

# Getting responses
response_original = get_response(question_original)
response_general = get_response(question_general)
response_engineered = get_response(question_engineered)

print("Original Prompt Response:", response_original)
print("General Prompt Response:", response_general)
print("Engineered Prompt Response:", response_engineered)

In [ ]:
# # Load the dataset
# df = pd.read_csv('result.csv')

# # Select columns for Zero-Shot CoT and Manual CoT methods
# cot_data = df[['question', 'zero_shot_cot_output', 'manual_cot_output']]

# # Process the data to create a combined prompt for each method
# cot_data['zero_shot_cot_prompt'] = cot_data.apply(lambda row: f"Question: {row['question']}\nAnswer: {row['zero_shot_cot_output']}", axis=1)
# cot_data['manual_cot_prompt'] = cot_data.apply(lambda row: f"Question: {row['question']}\nAnswer: {row['manual_cot_output']}", axis=1)

# # Combine the prompts into one dataset for tuning
# combined_prompts = pd.concat([cot_data['zero_shot_cot_prompt'], cot_data['manual_cot_prompt']])

In [4]:
def query_model(prompt):
    try:
        response = openai.Completion.create(
            model="text-davinci-003",  # Update the model as needed
            prompt=prompt,
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return str(e)

In [5]:
for index, row in df.iterrows():
    best_cot_method = 'zero_shot_output' 

    # Original question
    question = row['question']
    
    # Response from the best CoT method
    best_response = row[best_cot_method]

    # Create a refined prompt based on the best response
    refined_prompt = "Based on the following information, answer the question: " + best_response + " Question: " + question

    # Test the refined prompt with the model
    model_response = query_model(refined_prompt)

    # Output the refined prompt and model response for analysis
    print(f"Refined Prompt: {refined_prompt}")
    print(f"Model Response: {model_response}")

    # Update the DataFrame with the refined prompt and model response
    df.at[index, 'refined_prompt'] = refined_prompt
    df.at[index, 'model_response'] = model_response

Refined Prompt: Based on the following information, answer the question: yes. Question: Does the individual often have difficulty sustaining attention in tasks or play activities?
Model Response: You exceeded your current quota, please check your plan and billing details.
Refined Prompt: Based on the following information, answer the question: yes. Question: Is there a tendency to make careless mistakes in schoolwork or other activities due to lack of attention?
Model Response: You exceeded your current quota, please check your plan and billing details.
Refined Prompt: Based on the following information, answer the question: difficult to determine from the information given. Question: Does the person seem to be easily distracted by unrelated stimuli or events?
Model Response: You exceeded your current quota, please check your plan and billing details.
Refined Prompt: Based on the following information, answer the question: unclear. Question: Is there a frequent forgetfulness in daily a

Refined Prompt: Based on the following information, answer the question: yes. Question: Does the person often interrupt others' conversations or activities?
Model Response: You exceeded your current quota, please check your plan and billing details.
Refined Prompt: Based on the following information, answer the question: yes. Question: Is there a tendency to engage in risk-taking behaviors without considering consequences?
Model Response: You exceeded your current quota, please check your plan and billing details.
Refined Prompt: Based on the following information, answer the question: not known. Question: Does the individual have a history of struggling with written communication, such as writing essays or reports?
Model Response: You exceeded your current quota, please check your plan and billing details.
Refined Prompt: Based on the following information, answer the question: yes. Question: Is there difficulty in adapting to changes in routines or unexpected events?
Model Response: 

In [ ]:
# Save the updated DataFrame to a new CSV file
df.to_csv('updated_result.csv', index=False)

In [ ]:
## Llama

In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import TextGenerationPipeline

# Load LLaMA model and tokenizer
model_name = "Llama-13B"  # Replace with the specific LLaMA model you're using
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# Define a pipeline for text generation
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer)

# Define your prompts
prompts = [
    "Translate the following sentence to French: '{}'",
    "Summarize the following text: '{}'",
    # Add more prompts as needed
]

# Generate embeddings for each prompt
prompt_embeddings = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model(**inputs)
    prompt_embeddings.append(outputs.last_hidden_state)

# Fine-tune the model using these embeddings with your task-specific data

# Example of using the fine-tuned model
test_input = "Translate the following sentence to French: 'Hello, how are you?'"
generated_text = pipeline(test_input)
print(generated_text)